
https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|offset%3D50&format=json

Better query:
https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|limit%3D2000&format=json

Much better query: https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|?DBLP%20Series%20ID|?Title|?Acronym|?DOI|limit%3D3000&format=json

# Downloading data

In [1]:
import requests
import json

In [2]:
r = requests.get('https://www.confident-conference.org/api.php?action=ask&query=[[Category:Event_Series]]|?DBLP%20Series%20ID|?Title|?Acronym|?DOI|limit%3D3000&format=json')

In [3]:
r.status_code

200

In [4]:
result = r.json()

In [5]:
result.keys()

dict_keys(['query'])

In [6]:
with open('ConfIDent.json','w') as fw:
    json.dump(result, fw)

# Creating Index

In [7]:
with open('ConfIDent.json','r') as fr:
    confident = json.load(fr)

In [8]:
c_results = confident["query"]["results"]
unique_confs = {}
for k, c_series in c_results.items():
    unique_confs[c_series["displaytitle"]] = k
    if len(c_series["printouts"]["Title"]) > 0:
        unique_confs[c_series["printouts"]["Title"][0]] = k
len(unique_confs)

3711

In [21]:
dblp_confs = {}
event2dblp = {}
for k, c_series in c_results.items():
    if len(c_series["printouts"]["DBLP Series ID"]) > 0:
        dblp_confs[c_series["printouts"]["DBLP Series ID"][0]] = k
        event2dblp[k] = c_series["printouts"]["DBLP Series ID"][0]
len(dblp_confs)

566

In [22]:
unique_confs

{'3-Länder-Korrosionstagung': 'Event Series:C36681aa-b39c-4882-b19f-8d22572ec6ec',
 '3DUI - IEEE Symposium on 3D User Interfaces': 'Event Series:3DUI',
 'IEEE Symposium on 3D User Interfaces': 'Event Series:3DUI',
 '4T Design and Design History Symposium': 'Event Series:3a137ceb-c53d-493d-b0dd-1bf93400de00',
 'AAAAECC - International Symposium on Applied Algebra, Algebraic Algorithms and Error-Correcting Codes': 'Event Series:AAAAECC',
 'International Symposium on Applied Algebra, Algebraic Algorithms and Error-Correcting Codes': 'Event Series:AAAAECC',
 'AAAI - Conference on Artificial Intelligence': 'Event Series:AAAI',
 'Conference on Artificial Intelligence': 'Event Series:AAAI',
 "AAAI Spring Symposium - AAAI's Spring Symposium Series": 'Event Series:AAAI Spring Symposia',
 "AAAI's Spring Symposium Series": 'Event Series:AAAI Spring Symposia',
 'AAC - Aachen Acoustics Colloquium': 'Event Series:AAC',
 'Aachen Acoustics Colloquium': 'Event Series:AAC',
 'AACL - International Americ

In [23]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = list(unique_confs.keys())

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

vector_size = embeddings.shape[1]

(3711, 384)


In [24]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(vector_size)   # build the index
print(index.is_trained)

True


In [25]:
index.add(embeddings)                  # add vectors to the index
print(index.ntotal)

3711


In [26]:
to_save = dict()
to_save["sentences"] = sentences
to_save["index"] = index
to_save["confs"] = unique_confs
to_save["conf_series"] = c_results
to_save["dblp_confs"] = dblp_confs
to_save["confids"] = {k:v["displaytitle"] for k, v in c_results.items()}
to_save["event2dblp"] = event2dblp
import pickle
with open('ConfIDent.pickle', 'wb') as handle:
    pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)